In [ ]:
#pip install lumibot timedelta alpaca-trade-api==3.1.1

In [ ]:
#pip install torch torchvision torchaudio transformers

In [2]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta
from finbert_utils import estimate_sentiment

API_KEY = ""
API_SECRET = ""
BASE_URL = "https://paper-api.alpaca.markets"

ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}
class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price,0)
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol,
                                 start=three_days_prior,
                                 end=today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()

        if cash > last_price:
            if sentiment == "positive" and probability > .999:
                if self.last_trade == "sell":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order)
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999:
                if self.last_trade == "buy":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "sell",
                    take_profit_price=last_price*.8,
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order)
                self.last_trade = "sell"

start_date = datetime(2020,1,1)
end_date = datetime(2023,12,31)
broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=broker,
                    parameters={"symbol":"SPY",
                                "cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)
# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()


Starting backtest for MLTrader...
Progress ||   0.58%  [Elapsed: 0:00:03 ETA: 0:10:30]2024-02-07 09:23:30,352: root: INFO: MLTrader : Trading iteration ended at 2024-02-07 09:23:30
Progress ||   0.59%  [Elapsed: 0:00:03 ETA: 0:11:06]2024-02-07 09:23:30,371: root: INFO: Current backtesting datetime 2020-01-09 15:55:00-05:00
Progress ||   0.59%  [Elapsed: 0:00:03 ETA: 0:11:08]2024-02-07 09:23:30,398: root: INFO: Current backtesting datetime 2020-01-09 16:00:00-05:00

Creating trades plot...

Creating indicators plot...

Creating tearsheet...
2024-02-07 09:23:30,568: root: ERROR: Not enough data to create a tearsheet, at least 2 days of data are required. Skipping


{'cagr': 0.0,
 'volatility': 0.0,
 'sharpe': 0,
 'max_drawdown': {'drawdown': 0.0,
  'date': Timestamp('2020-01-03 09:30:00-0500', tz='America/New_York')},
 'romad': 0,
 'total_return': 0.0}